In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm_notebook
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns             

sns.set_style("darkgrid")
sys.path.append('/home/robot-i9/Desktop/Anand/')
print(sys.path)

from NeuralBlocks.models.vgg import VGG

['/home/robot-i9/Desktop/Anand/NeuralBlocks/notebooks', '/home/robot-i9/Desktop/Anand/NeuralBlocks/notebooks', '', '/home/robot-i9/Downloads/pynaoqi-python2.7-2.5.5.5-linux64/lib/python2.7/site-packages', '/home/robot-i9/miniconda3/envs/main/lib/python37.zip', '/home/robot-i9/miniconda3/envs/main/lib/python3.7', '/home/robot-i9/miniconda3/envs/main/lib/python3.7/lib-dynload', '/home/robot-i9/miniconda3/envs/main/lib/python3.7/site-packages', '/home/robot-i9/miniconda3/envs/main/lib/python3.7/site-packages/IPython/extensions', '/home/robot-i9/.ipython', '/home/robot-i9/Desktop/Anand/']


In [2]:
torch.manual_seed(2456)
cudnn.deterministic = True
cudnn.benchmark = False
np.random.seed(2456)

NUM_EPOCH = 200
BATCH_SIZE = 128
CHECKPOINT_INTERVAL = 100
LRS = [0.0001, 0.001, 0.01]
NORMS =[None,'BN', 'SN', 'WN', 'MSN', 'MSNTReLU', 'WNTReLU']
DATA_PATH = "/home/robot-i9/Desktop/Anand/NeuralBlocks/data_utils/datasets/KMNIST/"
SAVE_PATH = "/home/robot-i9/Desktop/Anand/NeuralBlocks/experiments/KMNIST/"

os.makedirs(SAVE_PATH, exist_ok=True)

In [3]:
transform_train = transforms.Compose(
    [#transforms.RandomAffine(degrees=7, translate=(0.1, 0.1), scale=(0.95, 1.05)), 
     transforms.ToTensor(),
     transforms.Normalize((0.19036,), (0.34743,)),
    ])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.19036,), (0.34743,)),
    ])

trainset = torchvision.datasets.KMNIST(root=DATA_PATH, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

testset = torchvision.datasets.KMNIST(root=DATA_PATH, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [4]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        train_loss_log.append(train_loss/(batch_idx+1))
        train_acc_log.append( 100.*correct/total)
        
        if(batch_idx%CHECKPOINT_INTERVAL==0):
             print("Train Epoch [{:3d}/{:3d}]Batch [{:3d}/{:3d}] Loss: {:.3f} Acc {:.3f}%".format(epoch, NUM_EPOCH,batch_idx, len(trainloader),
                train_loss/(batch_idx+1), 100.*correct/total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            test_loss_log.append(test_loss/(batch_idx+1))
            test_acc_log.append( 100.*correct/total)
        
            if(batch_idx%CHECKPOINT_INTERVAL==0):
                print("Test Epoch [{:3d}/{:3d}]Batch [{:3d}/{:3d}] Loss: {:.3f} Acc {:.3f}%".format(epoch, NUM_EPOCH,batch_idx, len(testloader),
                test_loss/(batch_idx+1), 100.*correct/total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir(SAVE_PATH+'checkpoint'):
            os.mkdir(SAVE_PATH+'checkpoint')
        torch.save(state, SAVE_PATH+'checkpoint/ckpt.pth')
        best_acc = acc

In [5]:
best_acc = 0 
lr= LRS[0]
for norm in tqdm_notebook(NORMS):
    net = VGG(11,1, num_class=10, norm=norm).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=lr)

    train_loss_log =[]
    train_acc_log = []
    test_loss_log =[]
    test_acc_log =[]

    for epoch in tqdm_notebook(range(NUM_EPOCH)):
        train(epoch)
        test(epoch)

    np.save(SAVE_PATH+"VGG_Train_loss_{}_{}.npy".format(norm,lr), train_loss_log)  
    np.save(SAVE_PATH+"VGG_Test_loss_{}_{}.npy".format(norm,lr), test_loss_log)    
    np.save(SAVE_PATH+"VGG_Train_Acc_{}_{}.npy".format(norm,lr), train_acc_log)    
    np.save(SAVE_PATH+"VGG_Test_Acc_{}_{}.npy".format(norm,lr), test_acc_log)   
    del net

Train Epoch [  0/200]Batch [  0/469] Loss: 2.303 Acc 10.938%
Train Epoch [  0/200]Batch [100/469] Loss: 1.314 Acc 52.413%
Train Epoch [  0/200]Batch [200/469] Loss: 0.862 Acc 69.726%
Train Epoch [  0/200]Batch [300/469] Loss: 0.667 Acc 76.965%
Train Epoch [  0/200]Batch [400/469] Loss: 0.546 Acc 81.334%
Test Epoch [  0/200]Batch [  0/ 79] Loss: 0.336 Acc 89.062%
Saving..
Train Epoch [  1/200]Batch [  0/469] Loss: 0.135 Acc 96.094%
Train Epoch [  1/200]Batch [100/469] Loss: 0.107 Acc 96.805%
Train Epoch [  1/200]Batch [200/469] Loss: 0.103 Acc 96.879%
Train Epoch [  1/200]Batch [300/469] Loss: 0.099 Acc 97.015%
Train Epoch [  1/200]Batch [400/469] Loss: 0.095 Acc 97.138%
Test Epoch [  1/200]Batch [  0/ 79] Loss: 0.208 Acc 92.969%
Saving..
Train Epoch [  2/200]Batch [  0/469] Loss: 0.037 Acc 98.438%
Train Epoch [  2/200]Batch [100/469] Loss: 0.050 Acc 98.461%
Train Epoch [  2/200]Batch [200/469] Loss: 0.053 Acc 98.391%
Train Epoch [  2/200]Batch [300/469] Loss: 0.052 Acc 98.419%
Train Ep

Train Epoch [ 22/200]Batch [200/469] Loss: 0.004 Acc 99.876%
Train Epoch [ 22/200]Batch [300/469] Loss: 0.004 Acc 99.886%
Train Epoch [ 22/200]Batch [400/469] Loss: 0.006 Acc 99.852%
Test Epoch [ 22/200]Batch [  0/ 79] Loss: 0.242 Acc 95.312%
Train Epoch [ 23/200]Batch [  0/469] Loss: 0.038 Acc 99.219%
Train Epoch [ 23/200]Batch [100/469] Loss: 0.010 Acc 99.822%
Train Epoch [ 23/200]Batch [200/469] Loss: 0.008 Acc 99.845%
Train Epoch [ 23/200]Batch [300/469] Loss: 0.007 Acc 99.829%
Train Epoch [ 23/200]Batch [400/469] Loss: 0.008 Acc 99.825%
Test Epoch [ 23/200]Batch [  0/ 79] Loss: 0.235 Acc 96.094%
Train Epoch [ 24/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [ 24/200]Batch [100/469] Loss: 0.004 Acc 99.915%
Train Epoch [ 24/200]Batch [200/469] Loss: 0.004 Acc 99.895%
Train Epoch [ 24/200]Batch [300/469] Loss: 0.005 Acc 99.886%
Train Epoch [ 24/200]Batch [400/469] Loss: 0.004 Acc 99.897%
Test Epoch [ 24/200]Batch [  0/ 79] Loss: 0.320 Acc 96.875%
Train Epoch [ 25/200]Batch

Train Epoch [ 44/200]Batch [400/469] Loss: 0.001 Acc 99.975%
Test Epoch [ 44/200]Batch [  0/ 79] Loss: 0.272 Acc 97.656%
Train Epoch [ 45/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [ 45/200]Batch [100/469] Loss: 0.001 Acc 99.992%
Train Epoch [ 45/200]Batch [200/469] Loss: 0.001 Acc 99.988%
Train Epoch [ 45/200]Batch [300/469] Loss: 0.003 Acc 99.961%
Train Epoch [ 45/200]Batch [400/469] Loss: 0.003 Acc 99.953%
Test Epoch [ 45/200]Batch [  0/ 79] Loss: 0.179 Acc 97.656%
Train Epoch [ 46/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [ 46/200]Batch [100/469] Loss: 0.006 Acc 99.869%
Train Epoch [ 46/200]Batch [200/469] Loss: 0.006 Acc 99.868%
Train Epoch [ 46/200]Batch [300/469] Loss: 0.006 Acc 99.870%
Train Epoch [ 46/200]Batch [400/469] Loss: 0.006 Acc 99.869%
Test Epoch [ 46/200]Batch [  0/ 79] Loss: 0.168 Acc 98.438%
Train Epoch [ 47/200]Batch [  0/469] Loss: 0.004 Acc 100.000%
Train Epoch [ 47/200]Batch [100/469] Loss: 0.003 Acc 99.954%
Train Epoch [ 47/200]Bat

Train Epoch [ 67/200]Batch [100/469] Loss: 0.001 Acc 99.969%
Train Epoch [ 67/200]Batch [200/469] Loss: 0.001 Acc 99.984%
Train Epoch [ 67/200]Batch [300/469] Loss: 0.001 Acc 99.987%
Train Epoch [ 67/200]Batch [400/469] Loss: 0.001 Acc 99.986%
Test Epoch [ 67/200]Batch [  0/ 79] Loss: 0.256 Acc 95.312%
Train Epoch [ 68/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [ 68/200]Batch [100/469] Loss: 0.000 Acc 100.000%
Train Epoch [ 68/200]Batch [200/469] Loss: 0.002 Acc 99.942%
Train Epoch [ 68/200]Batch [300/469] Loss: 0.002 Acc 99.948%
Train Epoch [ 68/200]Batch [400/469] Loss: 0.002 Acc 99.957%
Test Epoch [ 68/200]Batch [  0/ 79] Loss: 0.315 Acc 96.875%
Train Epoch [ 69/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [ 69/200]Batch [100/469] Loss: 0.001 Acc 99.977%
Train Epoch [ 69/200]Batch [200/469] Loss: 0.002 Acc 99.969%
Train Epoch [ 69/200]Batch [300/469] Loss: 0.002 Acc 99.977%
Train Epoch [ 69/200]Batch [400/469] Loss: 0.001 Acc 99.977%
Test Epoch [ 69/200]Bat

Train Epoch [ 89/200]Batch [300/469] Loss: 0.011 Acc 99.821%
Train Epoch [ 89/200]Batch [400/469] Loss: 0.009 Acc 99.856%
Test Epoch [ 89/200]Batch [  0/ 79] Loss: 0.611 Acc 96.875%
Train Epoch [ 90/200]Batch [  0/469] Loss: 0.001 Acc 100.000%
Train Epoch [ 90/200]Batch [100/469] Loss: 0.002 Acc 99.977%
Train Epoch [ 90/200]Batch [200/469] Loss: 0.003 Acc 99.965%
Train Epoch [ 90/200]Batch [300/469] Loss: 0.006 Acc 99.930%
Train Epoch [ 90/200]Batch [400/469] Loss: 0.005 Acc 99.940%
Test Epoch [ 90/200]Batch [  0/ 79] Loss: 0.288 Acc 96.094%
Train Epoch [ 91/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [ 91/200]Batch [100/469] Loss: 0.001 Acc 99.985%
Train Epoch [ 91/200]Batch [200/469] Loss: 0.001 Acc 99.981%
Train Epoch [ 91/200]Batch [300/469] Loss: 0.001 Acc 99.982%
Train Epoch [ 91/200]Batch [400/469] Loss: 0.002 Acc 99.973%
Test Epoch [ 91/200]Batch [  0/ 79] Loss: 0.678 Acc 96.094%
Train Epoch [ 92/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [ 92/200]Bat

...

Train Epoch [112/200]Batch [300/469] Loss: 0.001 Acc 99.979%
Train Epoch [112/200]Batch [400/469] Loss: 0.001 Acc 99.975%
Test Epoch [112/200]Batch [  0/ 79] Loss: 0.148 Acc 97.656%
Train Epoch [113/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [113/200]Batch [100/469] Loss: 0.000 Acc 99.992%
Train Epoch [113/200]Batch [200/469] Loss: 0.000 Acc 99.992%
Train Epoch [113/200]Batch [300/469] Loss: 0.001 Acc 99.984%
Train Epoch [113/200]Batch [400/469] Loss: 0.001 Acc 99.986%
Test Epoch [113/200]Batch [  0/ 79] Loss: 0.205 Acc 96.875%
Train Epoch [114/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [114/200]Batch [100/469] Loss: 0.000 Acc 99.992%
Train Epoch [114/200]Batch [200/469] Loss: 0.000 Acc 99.996%
Train Epoch [114/200]Batch [300/469] Loss: 0.000 Acc 99.997%
Train Epoch [114/200]Batch [400/469] Loss: 0.001 Acc 99.984%
Test Epoch [114/200]Batch [  0/ 79] Loss: 0.191 Acc 97.656%
Train Epoch [115/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [115/200]Bat

Test Epoch [134/200]Batch [  0/ 79] Loss: 0.372 Acc 96.875%
Train Epoch [135/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [135/200]Batch [100/469] Loss: 0.000 Acc 100.000%
Train Epoch [135/200]Batch [200/469] Loss: 0.000 Acc 99.996%
Train Epoch [135/200]Batch [300/469] Loss: 0.001 Acc 99.984%
Train Epoch [135/200]Batch [400/469] Loss: 0.001 Acc 99.984%
Test Epoch [135/200]Batch [  0/ 79] Loss: 0.264 Acc 97.656%
Train Epoch [136/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [136/200]Batch [100/469] Loss: 0.000 Acc 99.992%
Train Epoch [136/200]Batch [200/469] Loss: 0.000 Acc 99.996%
Train Epoch [136/200]Batch [300/469] Loss: 0.000 Acc 99.990%
Train Epoch [136/200]Batch [400/469] Loss: 0.000 Acc 99.988%
Test Epoch [136/200]Batch [  0/ 79] Loss: 0.354 Acc 96.094%
Train Epoch [137/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [137/200]Batch [100/469] Loss: 0.002 Acc 99.946%
Train Epoch [137/200]Batch [200/469] Loss: 0.002 Acc 99.953%
Train Epoch [137/200]Ba

Train Epoch [157/200]Batch [200/469] Loss: 0.000 Acc 99.992%
Train Epoch [157/200]Batch [300/469] Loss: 0.000 Acc 99.995%
Train Epoch [157/200]Batch [400/469] Loss: 0.000 Acc 99.994%
Test Epoch [157/200]Batch [  0/ 79] Loss: 0.420 Acc 97.656%
Train Epoch [158/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [158/200]Batch [100/469] Loss: 0.003 Acc 99.969%
Train Epoch [158/200]Batch [200/469] Loss: 0.001 Acc 99.981%
Train Epoch [158/200]Batch [300/469] Loss: 0.001 Acc 99.979%
Train Epoch [158/200]Batch [400/469] Loss: 0.002 Acc 99.969%
Test Epoch [158/200]Batch [  0/ 79] Loss: 0.318 Acc 97.656%
Train Epoch [159/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [159/200]Batch [100/469] Loss: 0.006 Acc 99.930%
Train Epoch [159/200]Batch [200/469] Loss: 0.003 Acc 99.965%
Train Epoch [159/200]Batch [300/469] Loss: 0.003 Acc 99.951%
Train Epoch [159/200]Batch [400/469] Loss: 0.003 Acc 99.947%
Test Epoch [159/200]Batch [  0/ 79] Loss: 0.199 Acc 97.656%
Train Epoch [160/200]Batc

Test Epoch [179/200]Batch [  0/ 79] Loss: 0.336 Acc 97.656%
Train Epoch [180/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [180/200]Batch [100/469] Loss: 0.002 Acc 99.946%
Train Epoch [180/200]Batch [200/469] Loss: 0.003 Acc 99.946%
Train Epoch [180/200]Batch [300/469] Loss: 0.002 Acc 99.956%
Train Epoch [180/200]Batch [400/469] Loss: 0.002 Acc 99.965%
Test Epoch [180/200]Batch [  0/ 79] Loss: 0.353 Acc 96.094%
Train Epoch [181/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [181/200]Batch [100/469] Loss: 0.002 Acc 99.977%
Train Epoch [181/200]Batch [200/469] Loss: 0.001 Acc 99.988%
Train Epoch [181/200]Batch [300/469] Loss: 0.001 Acc 99.992%
Train Epoch [181/200]Batch [400/469] Loss: 0.001 Acc 99.990%
Test Epoch [181/200]Batch [  0/ 79] Loss: 0.329 Acc 97.656%
Train Epoch [182/200]Batch [  0/469] Loss: 0.000 Acc 100.000%
Train Epoch [182/200]Batch [100/469] Loss: 0.000 Acc 100.000%
Train Epoch [182/200]Batch [200/469] Loss: 0.000 Acc 99.996%
Train Epoch [182/200]Ba

AttributeError: 'MeanSpectralNormConv2d' object has no attribute 'weight'

In [ ]:
k = 0
plt.figure(figsize=(18, 15))
for i, lr in enumerate(tqdm_notebook(LRS)):    
    for j, norm in enumerate(tqdm_notebook([None,'BN', 'SN', 'MSN'])):
        print(k)
        train_loss_log = np.load(SAVE_PATH+"VGG_Train_loss_{}_{}.npy".format(norm,lr) )  
        test_loss_log = np.load(SAVE_PATH+"VGG_Test_loss_{}_{}.npy".format(norm,lr))    
        train_acc_log = np.load(SAVE_PATH+"VGG_Train_Acc_{}_{}.npy".format(norm,lr))    
        test_acc_log= np.load(SAVE_PATH+"VGG_Test_Acc_{}_{}.npy".format(norm,lr))   
        
        ax = plt.subplot(3,4,k+1)
        plt.plot(train_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Loss', fontsize=18)     
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+2)
        plt.plot(test_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Loss', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+3)
        plt.plot(train_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Accuracy', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
                
        ax = plt.subplot(3,4,k+4)
        plt.plot(test_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Accuracy', fontsize=18)        
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
    k+= 4
plt.tight_layout()
plt.savefig(SAVE_PATH+'Act_Norm_Results.pdf', dpi=400)

In [ ]:
k = 0
plt.figure(figsize=(18, 15))
for i, lr in enumerate(tqdm_notebook(LRS)):    
    for j, norm in enumerate(tqdm_notebook(['WN', 'SN', 'MSN'])):
        print(k)
        train_loss_log = np.load(SAVE_PATH+"VGG_Train_loss_{}_{}.npy".format(norm,lr) )  
        test_loss_log = np.load(SAVE_PATH+"VGG_Test_loss_{}_{}.npy".format(norm,lr))    
        train_acc_log = np.load(SAVE_PATH+"VGG_Train_Acc_{}_{}.npy".format(norm,lr))    
        test_acc_log= np.load(SAVE_PATH+"VGG_Test_Acc_{}_{}.npy".format(norm,lr))   
        
        ax = plt.subplot(3,4,k+1)
        plt.plot(train_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Loss', fontsize=18)     
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+2)
        plt.plot(test_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Loss', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+3)
        plt.plot(train_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Accuracy', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
                
        ax = plt.subplot(3,4,k+4)
        plt.plot(test_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Accuracy', fontsize=18)        
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
    k+= 4
plt.tight_layout()
plt.savefig(SAVE_PATH+'Weight_reparam_Results.pdf', dpi=400)

In [ ]:
k = 0
plt.figure(figsize=(18, 15))
for i, lr in enumerate(tqdm_notebook(LRS)):    
    for j, norm in enumerate(tqdm_notebook(['MSNTReLU', 'WNTReLU'])):
        print(k)
        train_loss_log = np.load(SAVE_PATH+"VGG_Train_loss_{}_{}.npy".format(norm,lr) )  
        test_loss_log = np.load(SAVE_PATH+"VGG_Test_loss_{}_{}.npy".format(norm,lr))    
        train_acc_log = np.load(SAVE_PATH+"VGG_Train_Acc_{}_{}.npy".format(norm,lr))    
        test_acc_log= np.load(SAVE_PATH+"VGG_Test_Acc_{}_{}.npy".format(norm,lr))   
        
        ax = plt.subplot(3,4,k+1)
        plt.plot(train_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Loss', fontsize=18)     
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+2)
        plt.plot(test_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Loss', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+3)
        plt.plot(train_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Accuracy', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
                
        ax = plt.subplot(3,4,k+4)
        plt.plot(test_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Accuracy', fontsize=18)        
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
    k+= 4
plt.tight_layout()
plt.savefig(SAVE_PATH+'Weight_reparam_act_Results.pdf', dpi=400)